In [ ]:
import os
import sys

import numpy as np
import pandas as pd 

import pickle

In [ ]:
df=pd.read_csv('Data/data_preprocessed.csv')

In [ ]:
strategy_col=['AdDigitalChannel','AudienceType','Spend']
df_bid=df[strategy_col]

In [ ]:
df_bid

In [ ]:
df_bid['R1']=1*df['WebVisit']+2*df['CollateralView']+3*df['ProductView']+4*df['FormComplete']

In [ ]:
df_bid['R2']=-0.00000065492*(df['VideoCompletes']+df['SocialLikes']+df['SocialShares'])+0.33536101450

In [ ]:
df_bid['R']=df_bid['R1']+df_bid['R2']

In [ ]:
ch_li=list(set(df_bid['AdDigitalChannel'].to_list()))
ch_n=[1,2,3]
ch_dict={}
for i in range(len(ch_n)):
    ch_dict[ch_li[i]]=ch_n[i]

aud_li=list(set(df_bid['AudienceType'].to_list()))
aud_n=[1,2,3,4,5]
aud_dict={}
for i in range(len(aud_n)):
    aud_dict[aud_li[i]]=aud_n[i]

In [ ]:
df_bid['AdDigitalChannel']=df_bid['AdDigitalChannel'].apply(lambda a:ch_dict[a])
df_bid['AudienceType']=df_bid['AudienceType'].apply(lambda a:aud_dict[a])

In [ ]:
with open('Data/Y_exp_cat.pkl','rb')as f: #  Y_exp_xg.pkl
    ctr=pickle.load(f)

In [ ]:
ctr=pd.DataFrame(ctr)

In [ ]:
df_bid['CTR']=ctr[1]

In [ ]:
df_bid['Impressions']=df['Impressions']

In [ ]:
df_bid.head(4)

In [ ]:
#optimization trick
# if spend is 0, our equation also become zero, meaning that there is no effect on the result.
# but the machine compute the zero spend data, so it create more cpu load. 
# thus we removed zero spend data from dataset, compute the optimization task.

In [ ]:
df_bid=df_bid[df_bid['Spend']!=0]

In [ ]:
df_bid=df_bid[df_bid['Impressions']!=0]

In [ ]:
df_bid=df_bid.reset_index(drop=True)
df_bid.shape

# gurobi work starts from here

In [ ]:
import gurobipy as gp
from gurobipy import GRB

In [ ]:
winning_li=[x if x ==0 else 1 for x in df_bid['Spend']]
idx_winning={}
for i in range(len(df_bid)):
    idx_winning[(df_bid['AdDigitalChannel'][i],df_bid['AudienceType'][i],df_bid['Impressions'][i])]=winning_li[i]

In [ ]:
idx_cost={}
for i in range(len(df_bid)):
    idx_cost[(df_bid['AdDigitalChannel'][i],df_bid['AudienceType'][i],df_bid['Impressions'][i])]=df_bid['Spend'][i]

In [ ]:
idx_utility={}
for i in range(len(df_bid)):
    idx_utility[(df_bid['AdDigitalChannel'][i],df_bid['AudienceType'][i],df_bid['Impressions'][i])]=df_bid['utility'][i]

In [ ]:
idx_ctr={}
for i in range(len(df_bid)):
    idx_ctr[(df_bid['AdDigitalChannel'][i],df_bid['AudienceType'][i],df_bid['Impressions'][i])]=df_bid['CTR'][i]

In [ ]:
ca_min={}
for i in range(len(df_bid)):
    ca_min[(df_bid['AdDigitalChannel'][i],df_bid['AudienceType'][i])]=20

In [ ]:
C_T_I,winning=gp.multidict(idx_winning)
C_T_I,cost=gp.multidict(idx_cost)
C_T_I,utility=gp.multidict(idx_utility)
C_T_I,ctr=gp.multidict(idx_ctr)
T_I,min_=gp.multidict(ca_min)
mo = gp.Model('RTB')

x=mo.addVars(C_T_I, vtype=GRB.BINARY,name='assign')

#min const
pub = {}
for t,i in ca_min:
    pub[t,i] = mo.addConstr(gp.quicksum(x[t,i,k] for tt,ii,k in C_T_I if (tt ==t and ii == i) ) >= 20, 
                                          name ='pub' + str(t) + ',' + str(i) )

#max const    
pub2 = {}
for t,i in ca_min:
    pub2[t,i] = mo.addConstr(gp.quicksum(x[t,i,k] for tt,ii,k in C_T_I if (tt ==t and ii == i) ) <= 500, 
                                          name ='pub142' + str(t) + ',' + str(i) )

#total const
max_bud = mo.addConstr(x.prod(cost) <= 1000, name='max_budget')

#min_bud = mo.addConstrs((x[i]*cost[i] >=20 for i in C_T_I), name='min_budget')
#min_bud = mo.addConstr(x.prod(cost) >= 20, name='min_budget')

#objective
obj= gp.quicksum(x[i]*utility[i]*winning[i]*ctr[i] for i in x)

#optimize
mo.setObjective(obj, GRB.MAXIMIZE)
mo.optimize()

In [ ]:
c_t_util={}
c_t_cost={}

for c,t,i in C_T_I:
    if x[c,t,i].x > 0:
        if c_t_util.get((c,t))==None:
            c_t_util[(c,t)]=0
        if c_t_cost.get((c,t))==None:
            c_t_cost[(c,t)]=0
        c_t_util[(c,t)]=c_t_util[(c,t)]+utility[c,t,i]*x[c,t,i].x
        c_t_cost[(c,t)]=c_t_cost[(c,t)]+cost[c,t,i]*x[c,t,i].x
        

In [ ]:
result = pd.DataFrame(columns=["chtype", "audtype", "utility",'cost'])
for key in c_t_util.keys():
    result=result.append({"chtype": key[0], "audtype": key[1], "utility": c_t_util[key],"cost":c_t_cost[key]}, ignore_index=True)  


In [ ]:
def get_keys_from_value(d, val):
    output=[k for k, v in d.items() if v == val][0]
    return output

In [ ]:
result['chtype']=result['chtype'].apply(lambda x : get_keys_from_value(ch_dict,x))
result['audtype']=result['audtype'].apply(lambda x : get_keys_from_value(aud_dict,x))
result

In [ ]:
print('the entire cost is {}'.format(sum(result['cost'].tolist())))